### Imports

In [5]:
import pandas as pd
import numpy as np

### Data Cleaning

Finding where only Canada values are missing and filling those values with the mean value of all the major cities.

In [50]:
df = pd.read_csv(r'data/gas_data_table.csv')
df.replace('..', np.nan, inplace=True)

df.head()

,Geography,Jan-79,Feb-79,Mar-79,Apr-79,May-79,Jun-79,Jul-79,Aug-79,Sep-79,...,Apr-23,May-23,Jun-23,Jul-23,Aug-23,Sep-23,Oct-23,Nov-23,Dec-23,Jan-24
0,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,160.1,158.8,161.6,163.1,170.6,168.3,157.4,152.2,145.4,144.1
1,"St. John's, Newfoundland and Labrador",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,174.3,165.5,168.4,174.8,188.0,190.0,173.9,168.3,163.0,161.2
2,"Charlottetown and Summerside, Prince Edward Is...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,168.4,161.2,163.4,173.7,185.3,184.8,165.7,161.3,158.0,156.6
3,"Halifax, Nova Scotia",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,163.2,151.4,153.1,174.7,185.0,182.2,166.3,161.6,154.3,154.5
4,"Saint John, New Brunswick",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,169.3,159.1,161.4,170.6,182.7,184.5,167.3,160.3,154.8,152.7


In [64]:
df_clean = df 

# Convert columns to numeric, except the 'Geography' column
for column in df_clean.columns[1:]:  # Skip 'Geography' column
    df_clean[column] = pd.to_numeric(df_clean[column], errors='coerce')


# Identify the "Canada" row index for later use
canada_index = df_clean[df_clean['Geography'] == 'Canada'].index[0]

# Loop through columns starting from the third column (skipping 'Geography') to find the first eligible column
for column in df_clean.columns[2:]:
    # Check if "Canada" is missing and no other rows are missing in this column
    if pd.isna(df_clean.at[canada_index, column]) and df_clean[column].isna().sum() == 1:
        first_missing_canada_column = column
        break

# For columns where "Canada" is missing, fill with the mean of other values rounded to 1 decimal place
for column in df_clean.columns[2:]:
    if pd.isna(df_clean.at[canada_index, column]):
        # Calculate mean of other values in the column, excluding Canada
        mean_value = df_clean.loc[df_clean.index != canada_index, column].mean()
        # Fill the "Canada" cell with the mean, rounded to 1 decimal place
        df_clean.at[canada_index, column] = round(mean_value, 1)

In [62]:
# Keep only columns where all values are populated (no NaN values)
df_clean_columns_complete = df_clean.dropna(axis='columns')

# Display the resulting DataFrame to verify that only completely populated columns are kept
df_clean_columns_complete.head()

,Geography,Nov-92,Dec-92,Jan-93,Feb-93,Mar-93,Apr-93,May-93,Jun-93,Jul-93,...,Apr-23,May-23,Jun-23,Jul-23,Aug-23,Sep-23,Oct-23,Nov-23,Dec-23,Jan-24
0,Canada,56.1,56.1,55.9,54.9,54.0,54.1,54.5,55.5,55.2,...,160.1,158.8,161.6,163.1,170.6,168.3,157.4,152.2,145.4,144.1
1,"St. John's, Newfoundland and Labrador",59.4,59.2,57.1,57.1,56.4,56.4,57.4,58.4,59.5,...,174.3,165.5,168.4,174.8,188.0,190.0,173.9,168.3,163.0,161.2
2,"Charlottetown and Summerside, Prince Edward Is...",60.9,60.9,58.9,58.9,58.9,56.4,55.9,55.6,56.2,...,168.4,161.2,163.4,173.7,185.3,184.8,165.7,161.3,158.0,156.6
3,"Halifax, Nova Scotia",57.3,56.0,52.2,50.9,49.2,49.6,51.3,51.8,53.6,...,163.2,151.4,153.1,174.7,185.0,182.2,166.3,161.6,154.3,154.5
4,"Saint John, New Brunswick",56.1,55.3,54.6,54.5,54.5,54.4,54.7,54.8,55.7,...,169.3,159.1,161.4,170.6,182.7,184.5,167.3,160.3,154.8,152.7


### Writing to a cleaned csv

In [66]:
df_clean_columns_complete.to_csv(r'data/gas_data_clean.csv')